In [8]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms

In [2]:
path='/media/rohit/mirlproject2/rsna-breast-cancer-detection'

In [3]:
train_df = pd.read_csv('./train.csv')

In [4]:
train_df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [5]:
ddsm = train_df[['patient_id', 'image_id', 'cancer']]

In [6]:
img_folder= './archive (1)/train_images/'
ddsm['img_path_list'] = ddsm.apply(lambda x: img_folder+str(x.patient_id)+'/'+str(x.image_id)+'.png', axis = 1)


ddsm.tail(15)

/tmp/ipykernel_118709/4225720695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddsm['img_path_list'] = ddsm.apply(lambda x: img_folder+str(x.patient_id)+'/'+str(x.image_id)+'.png', axis = 1)


,patient_id,image_id,cancer,img_path_list
54691,9968,2098937312,0,./archive (1)/train_images/9968/2098937312.png
54692,9968,294168046,0,./archive (1)/train_images/9968/294168046.png
54693,9968,568005453,0,./archive (1)/train_images/9968/568005453.png
54694,997,873425464,0,./archive (1)/train_images/997/873425464.png
54695,997,2129119612,0,./archive (1)/train_images/997/2129119612.png
54696,997,1487679918,0,./archive (1)/train_images/997/1487679918.png
54697,997,1775983513,0,./archive (1)/train_images/997/1775983513.png
54698,9973,1345265157,0,./archive (1)/train_images/9973/1345265157.png
54699,9973,1703611570,0,./archive (1)/train_images/9973/1703611570.png
54700,9973,655941939,0,./archive (1)/train_images/9973/655941939.png


In [7]:
print(ddsm.iloc[0])
ddsm.iloc[0]['img_path_list']

patient_id                                                10006
image_id                                              462822612
cancer                                                        0
img_path_list    ./archive (1)/train_images/10006/462822612.png
Name: 0, dtype: object


'./archive (1)/train_images/10006/462822612.png'

In [9]:
from PIL import Image

img = Image.open(ddsm.iloc[17]['img_path_list'])
img=transforms.ToTensor()(img)
img.shape

torch.Size([1, 1024, 512])

In [10]:
images_list = ddsm['img_path_list']
images_list=list(images_list)
len(images_list)

54706

In [11]:
ddsm.sort_values('cancer', inplace =True)
ddsm

/tmp/ipykernel_118709/948297150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddsm.sort_values('cancer', inplace =True)


,patient_id,image_id,cancer,img_path_list
0,10006,462822612,0,./archive (1)/train_images/10006/462822612.png
36258,48921,1920949802,0,./archive (1)/train_images/48921/1920949802.png
36259,48939,105720752,0,./archive (1)/train_images/48939/105720752.png
36260,48939,441053398,0,./archive (1)/train_images/48939/441053398.png
36263,4894,368240487,0,./archive (1)/train_images/4894/368240487.png
...,...,...,...,...
17738,28747,1026238054,1,./archive (1)/train_images/28747/1026238054.png
17739,28747,2075658562,1,./archive (1)/train_images/28747/2075658562.png
29638,41610,1957729736,1,./archive (1)/train_images/41610/1957729736.png
8602,19182,1872087937,1,./archive (1)/train_images/19182/1872087937.png


In [12]:
ddsm['cancer'].sum()

1158

In [13]:
paths = ddsm["img_path_list"]
labels = ddsm["cancer"]
from sklearn.model_selection import train_test_split
paths_train,paths_test,labels_train, labels_test = train_test_split(paths, labels, stratify = labels, random_state = 42, test_size = 0.10)

In [14]:
paths_train,paths_val, labels_train, labels_val = train_test_split(paths_train,labels_train, stratify = labels_train, random_state = 42, test_size =0.11)

In [15]:
df1 = pd.DataFrame()
df1["img_path"] = paths_train
df1["labels"] = labels_train
df1.to_csv("./csv/train.csv")
df2 = pd.DataFrame()
df2["img_path"] = paths_val
df2["labels"] = labels_val
df2.to_csv("./csv/val.csv")
df3 = pd.DataFrame()
df3["img_path"] = paths_test
df3["labels"] = labels_test
df3.to_csv("./csv/test.csv")

In [16]:
labels.value_counts()

0    53548
1     1158
Name: cancer, dtype: int64

In [17]:
labels_train.value_counts()

0    42892
1      927
Name: cancer, dtype: int64

In [18]:
labels_val.value_counts()

0    5301
1     115
Name: cancer, dtype: int64

In [19]:
labels_test.value_counts()

0    5355
1     116
Name: cancer, dtype: int64

In [20]:
df3

,img_path,labels
48370,./archive (1)/train_images/62043/1040092701.png,0
24511,./archive (1)/train_images/3625/2045356515.png,0
14646,./archive (1)/train_images/25630/762192354.png,0
49944,./archive (1)/train_images/63884/2074069873.png,0
11248,./archive (1)/train_images/22060/1790151672.png,0
...,...,...
41926,./archive (1)/train_images/55247/2132649741.png,0
611,./archive (1)/train_images/10638/1696075167.png,1
41669,./archive (1)/train_images/55010/733599660.png,0
12179,./archive (1)/train_images/23013/1514069703.png,0
